In [15]:
#!/usr/bin/env python
"""
"""
import json
import logging
import re
import os
import uuid
from osgeo import ogr
from osgeo import osr


from collections import defaultdict
from datetime import datetime, timedelta
from pathlib import Path

import netCDF4
import numpy as np
import rasterio

from datacube import Datacube
from datacube.index.hl import Doc2Dataset
from datacube.utils import changes

os.environ['GDAL_NETCDF_BOTTOMUP'] = 'NO'

barra_variables = [
    {
        'barra_var_name':'av_wndgust10m',
        'barra_description':'Gust windspeed at 10m above-ground-level.',
        'barra_unit':'m s-1',
        'barra_path':'/g/data/ma05/BARRA_R/v1/forecast/slv/av_wndgust10m',
        'product_name':'barra_av_wndgust10m'
    },
    {
        'barra_var_name':'av_uwnd10m',
        'barra_description':'Zonal (U) component of the wind velocity at 10m above-ground-level.',
        'barra_unit':'m s-1',
        'barra_path':'/g/data/ma05/BARRA_R/v1/forecast/slv/av_uwnd10m',
        'product_name':'barra_av_uwnd10m'
    },
    {
        'barra_var_name':'av_vwnd10m',
        'barra_description':'Meridional (V) component of the wind velocity at 10m above-ground-level.',
        'barra_unit':'m s-1',
        'barra_path':'/g/data/ma05/BARRA_R/v1/forecast/slv/av_vwnd10m',
        'product_name':'barra_av_vwnd10m'
    }
]

LOG = logging.getLogger(__name__)

def print_dict(doc):
    print(json.dumps(doc, indent=4, sort_keys=True, cls=NumpySafeEncoder))

def find_datasets(barra_variable):
    """
    """
    path = Path(barra_variable['barra_path'])
    path = sorted(path.glob('**/*.nc'))
    ## TODO take into account variable type
    pattern = re.compile(r'(?P<barra_var>'+barra_variable['barra_var_name']+')\-(?P<barra_var_type>fc\-slv)\-(?P<barra_sampling_frequency>PT1H)\-(?P<barra_domain>BARRA_R)\-v(?P<barra_version>\d{1}\.?\d?)\-(?P<date>\d{8})T(?P<time>\d{4})Z\.sub.nc')
    datasets = defaultdict(dict)
    for ncfile in path:
        match = pattern.search(str(ncfile))
        if match:
            barra_var, barra_var_type, barra_sampling_frequency, barra_domain, barra_version, date, hour = match.groups()
            dataset = barra_var + date + hour+ barra_domain + barra_version
            datasets[dataset][barra_var] = ncfile
    return datasets

def generate_product_defn(barra_variable):
    return {
        'name': barra_variable['barra_var_name'],
        'metadata_type': 'eo',
        'metadata': {
            'product_type': barra_variable['product_name'],
            'format' : { 'name': 'NetCDF'}
        },
        'storage': {
            'crs': 'GEOGCS["unknown",DATUM["unknown",SPHEROID["Sphere",6371229,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]]',
            'resolution': {
                'latitude': 0.1100000035809413773,
                'longitude': 0.1100000058540808734
            },
            'origin': { 'longitude':65, 'latitude':19.48}
        },
        'description': barra_variable['barra_description'],
        'measurements': [
            {
                'name': barra_variable['barra_var_name'],
                'dtype':'float64',
                'nodata': -1073741824,
                'units':barra_variable['barra_unit']
            }
        ]
    }

def generate_dataset_docs(dataset_name, dataset, barra_variable):
    """
    """
    barra_var_name = barra_variable['barra_var_name']
    sample_ncfile = dataset[barra_var_name]
    sample_ncfile_gdal = f'NETCDF:{sample_ncfile}:{barra_var_name}'

    creation_time = datetime.fromtimestamp(sample_ncfile.stat().st_mtime)
    geo_ref_points, spatial_ref = get_grid_spatial_projection(
        sample_ncfile_gdal)
    name_offset = len(barra_var_name)
    date_length = 12
    date = name[name_offset:name_offset+date_length]

    start_time = datetime.strptime(date, '%Y%m%d%H%M')
    end_time = start_time + timedelta(hours=1) - timedelta(microseconds=1)
    center_time = start_time + timedelta(minutes=30)
    docs = []
    for i in range(6):
        unique_ds_uri = f'{sample_ncfile.as_uri()}#{creation_time}#{start_time+timedelta(hours=i)}'
        doc = {
            'id': str(uuid.uuid5(uuid.NAMESPACE_URL, unique_ds_uri)),
            'product_type': barra_variable['product_name'],
            'creation_dt': str(creation_time),
            'extent': {
                'from_dt': str(start_time+timedelta(hours=i)),
                'to_dt': str(end_time+timedelta(hours=i)),
                'center_dt': str(center_time+timedelta(hours=i)),
                'coord': to_lat_long_extent(geo_ref_points),
            },
            'format': {'name': 'NetCDF'},
            'grid_spatial': {
                'projection': {
                    'geo_ref_points': geo_ref_points,
                    'spatial_reference': spatial_ref,
                }
            },
            'image': {
                'bands': {
                    barra_var_name: {
                        'path': '',
                        'layer': barra_var_name,
                    }
                }
            },
            'lineage': {'source_datasets': {}}
        }
        docs.append(('file:'+str(dataset[barra_var_name])+'#part='+str(i),doc))
    return docs


def to_lat_long_extent(geo_ref_points):
    return {corner: {'lat': points['y'], 'lon': points['x']}
for corner, points in geo_ref_points.items()}


def get_grid_spatial_projection(fname):
    with rasterio.open(fname, 'r') as img:
        left, bottom, right, top = img.bounds
        spatial_reference = str(str(getattr(img, 'crs_wkt', None) or img.crs.wkt))
        geo_ref_points = {
            'ul': {'x': left, 'y': top},
            'ur': {'x': right, 'y': top},
            'll': {'x': left, 'y': bottom},
            'lr': {'x': right, 'y': bottom},
        }
        return geo_ref_points, spatial_reference

In [6]:
dc = Datacube(config='datacube.conf',env='barra-dev')
index = dc.index


DatasetType(name='av_wndgust10m', id_=18)
DatasetType(name='av_uwnd10m', id_=19)
DatasetType(name='av_vwnd10m', id_=20)


In [ ]:
for barra_variable in barra_variables:
    product_def = generate_product_defn(barra_variable)
    product = index.products.from_doc(product_def)
    indexed_product = index.products.add(product)
    print(indexed_product)

In [16]:
for barra_variable in barra_variables:
    datasets = find_datasets(barra_variable)
    resolver = Doc2Dataset(index)

    for name, dataset in datasets.items():
        docs = generate_dataset_docs(name, dataset, barra_variable)
        for doc in docs:       
            dataset, err = resolver(doc[1], doc[0])
            try:
                indexed_dataset = index.datasets.add(dataset)
            except Exception as e:
                logging.error("Couldn't index %s/%s", path, name)
                logging.exception("Exception", e)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

